In [1]:
import os, nltk, codecs
from nltk import word_tokenize
from time import time
import sys
t0 = time()

# to use my own preprocessing in the vectorizer later
def my_dummy(doc):
    return doc

In [2]:
#Load Data to pandas frame vor better visualization
import pandas as pd
complete_data = pd.read_csv('omt_shortned.txt', sep = ";",
                           names = ["#","Answers", "Class" , "Subclass"] )
print('before cleaning...\n')
print(complete_data.info(), "\n")
print(complete_data["Class"].value_counts(), "\n")
#delete eventually empty entries
complete_data = complete_data.dropna(axis = 0)
#delete eventually wrong labels ('\N' seems to accure from time to time)
complete_data = complete_data.drop(complete_data[complete_data.Class == '\\N'].index).reset_index()
print('after cleaning...\n')
print(complete_data.info())
print(complete_data["Class"].value_counts(), "\n")
print(complete_data.head)

before cleaning...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
#           1000 non-null object
Answers     1000 non-null object
Class       1000 non-null object
Subclass    1000 non-null object
dtypes: object(4)
memory usage: 31.3+ KB
None 

M     518
L     194
A     192
0      91
\N      5
Name: Class, dtype: int64 

after cleaning...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 995 entries, 0 to 994
Data columns (total 5 columns):
index       995 non-null int64
#           995 non-null object
Answers     995 non-null object
Class       995 non-null object
Subclass    995 non-null object
dtypes: int64(1), object(4)
memory usage: 38.9+ KB
None
M    518
L    194
A    192
0     91
Name: Class, dtype: int64 

<bound method NDFrame.head of      index                               #  \
0        0        185124138106620081063215   
1        1        185124138106620081063215   
2        2        185124138106620081063215   
3

In [6]:
from sklearn.model_selection import train_test_split

# extract the needed infos from data_complete
data = complete_data.filter(["Answers", "Class"],axis = 1)

#split it into X ( = input) and y ( the labels)
X, y = data["Answers"], data["Class"]

#split data into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size = 0.1, random_state = 42)


#Bei größerem Trainingsset: stratified_shufflesplit nutzen für balancierte sets

In [7]:
#preprocessing
import re
from nltk.corpus import stopwords
import numpy as np

# The pattern I (maybe) don't want in my corpus 
pattern = re.compile("\-|\!|\:|\.|\d|\,|\:|\(|\)|\?|\"|\\|\+|\%|\/")
def remove_patterns_from_string(strg):
    return re.sub(pattern, " ", strg)

# transforms pattern like xyz.Abc to xyz. Abc (--> helps tokenizer to detect sentences)
def insert_space_between_special_chars(strg):
    return re.sub('([a-zA-Z])([^A-Za-z0-9]+)([a-zA-Z])', r'\1\2 \3', strg)

# german stopwords (words who accure very often in texts and don't contribute anything to the text like articles, conjunctions)
stopWords = set(stopwords.words("german"))

#new spellings
stopWords.add('dass')

#tokenizes a list of strings
#return: list of tokens
def tokenize_text(text):
    tokens = nltk.word_tokenize(text, language = 'german')
    return tokens

#retrns: list of strings which do not occure in stopWords
def remove_stopwords(text):
    return [w for w in text if not w in stopWords]

# in : row for row from vector
# out: preprocessed version of the row
def preprocess_text(corpus):
    normalized_corpus = []
    for text in corpus:
        #to change xyz.abc to xyz. abc
        text = re.sub('([a-zA-Z])(\.)([a-zA-Z])', r'\1\2 \3', text)
        text = text.lower()
        #text = remove_patterns_from_string(text)
        text = tokenize_text(text)
        text = remove_stopwords(text)
        normalized_corpus.append(text)
    return normalized_corpus

#later preprocess only X_train and X_val. X_test gets its own pipeline in the end
X_train_preprocessed = preprocess_text(X_train)
X_test_preprocessed = preprocess_text(X_test)


print(X_train_preprocessed[:2])
print(type(X_train_preprocessed))
print(type(y_train))

[['schweren', 'unfall', 'neurologischen', 'test', 'bestehen', '.', 'gespannt', 'aufgeregt', 'ängstlich', 'gut', 'test', 'meistern', '.', 'möchte', 'unfähig', 'halten', 'paar', 'schäden', 'unfall', 'zurückbleiben', '.', 'meistert', 'test', 'zufrieden', 'arbeit/unfallnachsorge', 'therapeuten', '.'], ['person', 'schimpft', ',', 'person', 'fehler', 'gemacht', '.', 'einerseits', 'überlegen', ',', 'andererseits', '``', 'oller', 'besserwisser', "''", '.weil', 'falle', 'verursacher', ',', 'genau', 'weiß', ',', 'hätte', 'passieren', '.', 'ende', 'sieht', 'fehlverhalten', ',', 'person', 'dennoch', 'gekränkt', '.']]
<class 'list'>
<class 'pandas.core.series.Series'>


In [8]:
from sklearn.metrics import classification_report
import pandas as pd

# minimal classification performance: all answers set to M
m_labels = []
for i in range(len(y_test)):
    m_labels.append("M")
machtmotiv = pd.Series(m_labels)
#plot_classification_report(classification_report(y_test, machtmotiv))
print(classification_report(y_test, machtmotiv))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        14
           A       0.00      0.00      0.00        14
           L       0.00      0.00      0.00        20
           M       0.52      1.00      0.68        52

   micro avg       0.52      0.52      0.52       100
   macro avg       0.13      0.25      0.17       100
weighted avg       0.27      0.52      0.36       100



In [11]:
#Test different classifiers from scikit with tfidf model
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB

from sklearn.feature_extraction.text import TfidfVectorizer
test_tfidf = TfidfVectorizer(norm='l2', smooth_idf = True, use_idf = True, ngram_range = (1,2), 
                                         analyzer = 'word', tokenizer = my_dummy,
                                         preprocessor = my_dummy, token_pattern = None)
X_train_tfidf = test_tfidf.fit_transform(X_train_preprocessed)
X_test_tfidf = test_tfidf.transform(X_test_preprocessed)

names = ['LogRegression',
         'RandomForest',
         'GradientBoosting',
         'KNN',
         'SVC_lin',
         'SVC_rbf',
         'MNB']
classifiers = [LogisticRegression(multi_class = 'ovr'),
             RandomForestClassifier(),
             GradientBoostingClassifier(n_estimators = 200, max_depth = 8),
             KNeighborsClassifier(),
             SVC(kernel="linear"),
              SVC(gamma='scale'),
              MultinomialNB(),  
             ]

for name, clf in zip(names, classifiers):
    
    
    time_test = time()
    clf.fit(X_train_tfidf, y_train)
    y_predicted_class_test = clf.predict(X_test_tfidf)
    class_rep = classification_report(y_test, y_predicted_class_test) 
    print(name)
    print()
    print("done in %0.3fs." % (time() - time_test))
    print()
    print(class_rep)
    print('\n\n\n')


C:\Users\fmeyer\Anaconda3\envs\classification_omt\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\fmeyer\Anaconda3\envs\classification_omt\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\fmeyer\Anaconda3\envs\classification_omt\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


LogRegression

done in 0.019s.

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        14
           A       1.00      0.21      0.35        14
           L       1.00      0.35      0.52        20
           M       0.58      1.00      0.73        52

   micro avg       0.62      0.62      0.62       100
   macro avg       0.64      0.39      0.40       100
weighted avg       0.64      0.62      0.53       100





RandomForest

done in 0.081s.

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        14
           A       0.47      0.64      0.55        14
           L       0.65      0.55      0.59        20
           M       0.67      0.81      0.73        52

   micro avg       0.62      0.62      0.62       100
   macro avg       0.45      0.50      0.47       100
weighted avg       0.54      0.62      0.58       100





GradientBoosting

done in 27.152s.

              precision

C:\Users\fmeyer\Anaconda3\envs\classification_omt\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


SVC_lin

done in 0.375s.

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        14
           A       0.69      0.64      0.67        14
           L       0.82      0.45      0.58        20
           M       0.63      0.92      0.75        52

   micro avg       0.66      0.66      0.66       100
   macro avg       0.54      0.50      0.50       100
weighted avg       0.59      0.66      0.60       100





SVC_rbf

done in 0.336s.

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        14
           A       0.00      0.00      0.00        14
           L       0.00      0.00      0.00        20
           M       0.52      1.00      0.68        52

   micro avg       0.52      0.52      0.52       100
   macro avg       0.13      0.25      0.17       100
weighted avg       0.27      0.52      0.36       100





MNB

done in 0.008s.

              precision    recall  f1-score   su

C:\Users\fmeyer\Anaconda3\envs\classification_omt\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [9]:
# This cell trains SVC classifier with different features (tf, tfidf, min/max _docfrequency, kernels, C, iterations )
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn. decomposition import LatentDirichletAllocation
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline, FeatureUnion


print("start classification with SVC...")
#define pipeline combining feature extractor with classifier
pipeline1 = Pipeline([
    ('vect1', CountVectorizer(preprocessor = my_dummy,
                              tokenizer =my_dummy, 
                              token_pattern=None)),
                            
    ('tf1',TfidfTransformer()),
                            
    ('clf1', SVC()),    
])

parameters1 = {
    'vect1__max_df': (0.9, 1.0),
    'vect1__ngram_range':((1,1),(1,2),(1,3)),
    'vect1__min_df': (1,2),
    'tf1__use_idf': (True, False),
    'clf1__kernel': ('linear','rbf'), 
    'clf1__C':(0.001,0.1,1,10,100),
    'clf1__max_iter':(50,100,200),
}

if __name__ == "__main__":
    #find best params for this feature extraction methods
    grid_search1 = GridSearchCV(pipeline1, parameters1, cv=5,
                           n_jobs = -1, verbose = 1)
    print("performing gridsearch...")
    print("pipeline:", [name for name, _ in pipeline1.steps])
    print("parameters:")
    print(parameters1)
    time_svc = time()
    grid_search1.fit(X_train_preprocessed, y_train)
    print("done in %0.3fs" % (time()-time_svc))
    print()

    print("Best score: %0.3f" % grid_search1.best_score_)
    print("best parameters set:")
    best_parameters = grid_search1.best_estimator_.get_params()
    for param_name in sorted(parameters1.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

start classification with SVC...
performing gridsearch...
pipeline: ['vect1', 'tf1', 'clf1']
parameters:
{'vect1__max_df': (0.9, 1.0), 'vect1__ngram_range': ((1, 1), (1, 2), (1, 3)), 'vect1__min_df': (1, 2), 'tf1__use_idf': (True, False), 'clf1__kernel': ('linear', 'rbf'), 'clf1__C': (0.001, 0.1, 1, 10, 100), 'clf1__max_iter': (50, 100, 200)}
Fitting 5 folds for each of 720 candidates, totalling 3600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   18.2s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   43.9s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 3600 out of 3600 | elapsed:  6.5min finished
C:\Users\fmeyer\Anaconda3\envs\classification_omt\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


done in 391.549s

Best score: 0.706
best parameters set:
	clf1__C: 1
	clf1__kernel: 'linear'
	clf1__max_iter: 200
	tf1__use_idf: True
	vect1__max_df: 0.9
	vect1__min_df: 1
	vect1__ngram_range: (1, 2)


C:\Users\fmeyer\Anaconda3\envs\classification_omt\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn. decomposition import LatentDirichletAllocation
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion


print("start classification with RandomForest...")
#define pipeline combining feature extractor with classifier
pipeline2 = Pipeline([
    ('vect2', CountVectorizer(preprocessor = my_dummy,
                              tokenizer =my_dummy, 
                              token_pattern=None)),
                            
                            
    ('clf2', RandomForestClassifier()),    
])

parameters2 = {
    'vect2__max_df': (0.9,0.95,1.0),
    'vect2__ngram_range':((1,1),(1,3)),
    'vect2__min_df': (2,5), 
    'clf2__n_estimators':(10,100,200), 
    'clf2__criterion': ('gini','entropy'),
}

if __name__ == "__main__":
    #find best params for this feature extraction methods
    grid_search2 = GridSearchCV(pipeline2, parameters2, cv=5,
                           n_jobs = -1, verbose = 1)
    print("performing gridsearch...")
    print("pipeline:", [name for name, _ in pipeline2.steps])
    print("parameters:")
    print(parameters2)
    time_svc = time()
    grid_search2.fit(X_train_preprocessed, y_train)
    print("done in %0.3fs" % (time()-time_svc))
    print()

    print("Best score: %0.3f" % grid_search2.best_score_)
    print("best parameters set:")
    best_parameters = grid_search2.best_estimator_.get_params()
    for param_name in sorted(parameters2.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

start classification with RandomForest...
performing gridsearch...
pipeline: ['vect2', 'clf2']
parameters:
{'vect2__max_df': (0.9, 0.95, 1.0), 'vect2__ngram_range': ((1, 1), (1, 3)), 'vect2__min_df': (2, 5), 'clf2__n_estimators': (10, 100, 200), 'clf2__criterion': ('gini', 'entropy')}
Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   22.0s
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:   43.5s finished
C:\Users\fmeyer\Anaconda3\envs\classification_omt\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


done in 44.158s

Best score: 0.678
best parameters set:
	clf2__criterion: 'gini'
	clf2__n_estimators: 200
	vect2__max_df: 1.0
	vect2__min_df: 5
	vect2__ngram_range: (1, 1)


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn. decomposition import LatentDirichletAllocation
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC
from scipy.stats import randint as sp_randint

print("start classification with SVC and extra LDA - Features...")
#define pipeline combining feature extractor with classifier
pipeline5 = Pipeline([
    ('union1', FeatureUnion(n_jobs = -1,
                          transformer_list = [
                              ('vect3', CountVectorizer(preprocessor = my_dummy,
                                                        tokenizer =my_dummy, 
                                                        token_pattern=None)),
                                  
                            
    
                              
                              ('pipe1', Pipeline([
                                  ('vect_lda1', CountVectorizer(preprocessor = my_dummy,
                                                               tokenizer =my_dummy, token_pattern=None)),
                                   ('lda1', LatentDirichletAllocation(max_iter = 100, learning_method = 'online'))
                              ]))
                            ])),
                            
    ('clf5', SVC()),    
])

parameters5 = {
    'union1__vect3__max_df': [0.9,1.0],
    'union1__vect3__ngram_range':[(1,1),(1,3)],
    'union1__vect3__min_df': sp_randint(1,10),
    'union1__pipe1__vect_lda1__ngram_range':[(1,1)],
    'union1__pipe1__lda1__n_components': [3,10],
    'union1__pipe1__lda1__learning_offset': [5],
    'clf5__kernel': ('linear',), 
    'clf5__C': [0.01,0.1,1,10],
    'clf5__max_iter': [100],
    }

if __name__ == "__main__":
    #find best params for this feature extraction methods
    randomizedSearch1 = RandomizedSearchCV(pipeline5, n_iter = 10,
                                           param_distributions =parameters5, cv=3,
                                           n_jobs = -1,)
    print("performing randomizedsearch...")
    print("pipeline:", [name for name, _ in pipeline5.steps])
    print("parameters:")
    print(parameters5)
    time_svc = time()
    randomizedSearch1.fit(X_train_preprocessed, y_train)
    print("done in %0.3fs" % (time()-time_svc))
    print()

    print("Best score: %0.3f" % randomizedSearch1.best_score_)
    print("best parameters set:")
    best_parameters = randomizedSearch1.best_estimator_.get_params()
    for param_name in sorted(parameters5.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

start classification with SVC and extra LDA - Features...
performing randomizedsearch...
pipeline: ['union1', 'clf5']
parameters:
{'union1__vect3__max_df': [0.9, 1.0], 'union1__vect3__ngram_range': [(1, 1), (1, 3)], 'union1__vect3__min_df': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000018E2F57A9E8>, 'union1__pipe1__vect_lda1__ngram_range': [(1, 1)], 'union1__pipe1__lda1__n_components': [3, 10], 'union1__pipe1__lda1__learning_offset': [5], 'clf5__kernel': ('linear',), 'clf5__C': [0.01, 0.1, 1, 10], 'clf5__max_iter': [100]}
